In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

In [4]:
resnet = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(256, 256, 3)
)

resnet.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
layer_names = [
    "conv2_block3_out",
    "conv3_block4_out",
    "conv4_block6_out"
]

outputs = [resnet.get_layer(name).output for name in layer_names]

backbone = Model(
    inputs=resnet.input,
    outputs=outputs,
    name="patchcore_backbone"
)

In [6]:
TARGET_SIZE = (64, 64)

In [7]:
def align_features(feature_maps, target_size = (64, 64)):
  aligned =[]

  for f in feature_maps:
    resized = tf.image.resize(
        f,
        size = TARGET_SIZE,
        method = "bilinear"
    )

    aligned.append(resized)
  return aligned

In [8]:
def concatenate_features(aligned_features):
    return tf.concat(aligned_features, axis=-1)

In [9]:
dummy = tf.random.uniform((1, 256, 256, 3))

features = backbone(dummy, training=False)
aligned = align_features(features)
patch_embedding = concatenate_features(aligned)

print("Final patch embedding shape:", patch_embedding.shape)

Final patch embedding shape: (1, 64, 64, 1792)


In [10]:
def extract_patches(feature_maps):
  """
  feature vector : (1, 64, 64, 1792)
  return : (B*H*W, C)
  """
  b, h, w, c = feature_maps.shape
  patches = tf.reshape(feature_maps, (-1, c))
  return patches

In [11]:
def build_memory_bank(patches, sampling_ratio=0.1):
    """
    patches: (N, C)
    sampling_ratio: % patches to keep
    """
    patches = patches / np.linalg.norm(patches, axis=1, keepdims=True)
    n_total = patches.shape[0]
    n_sample = int(n_total * sampling_ratio)

    indices = np.random.choice(n_total, n_sample, replace=False)
    memory_bank = patches[indices]

    return memory_bank

In [12]:
from sklearn.metrics import pairwise_distances

def compute_patch_score(test_patch, memory_bank):
  """
  test_patches: (N, C)
  memory_bank: (M, C)
  returns: (N,) anomaly score per patch
  """

  distance = pairwise_distances(test_patch, memory_bank)
  patch_score = distance.min(axis = 1)
  return patch_score

In [13]:
def image_anomaly_score(patch_scores):
    return patch_scores.max()

In [14]:
def build_anomaly_map(patch_scores, h=64, w=64):
    return patch_scores.reshape(h, w)